In [5]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import polars as pl


path2add = os.path.normpath(os.path.abspath(os.path.join(os.path.dirname('__file__'), os.path.pardir, 'utils')))
if (not (path2add in sys.path)) :
    sys.path.append(path2add)

import feature_engineering


In [15]:
df = pl.read_csv("..\data\cleaned_df.csv")
callGraph = pl.read_json('..\data\supervised_call_graphs.json')

In [14]:
df.head(3)

,_id,inter_api_access_duration(sec),api_access_uniqueness,sequence_length(count),vsession_duration(min),ip_type,num_sessions,num_users,num_unique_apis,source,classification,target
i64,str,f64,f64,f64,i64,str,f64,f64,f64,str,str,i64
0,"""1f2c32d8-2d6e-…",0.000812,0.004066,85.643243,5405,"""default""",1460.0,1295.0,451.0,"""E""","""normal""",0
1,"""4c486414-d4f5-…",0.000063,0.002211,16.166805,519,"""default""",9299.0,8447.0,302.0,"""E""","""normal""",0
2,"""7e5838fc-bce1-…",0.004481,0.015324,99.573276,6211,"""default""",255.0,232.0,354.0,"""E""","""normal""",0


In [16]:
callGraph.head(3)

_id,call_graph
str,list[struct[2]]
"""1f2c32d8-2d6e-…","[{""1f873432-6944-3df9-8300-8a3cf9f95b35"",""5862055b-35a6-316a-8e20-3ae20c1763c2""}, {""8955faa9-0e33-37ad-a1dc-f0e640a114c2"",""a4fd6415-1fd4-303e-aa33-bb1830b5d9d4""}, … {""016099ea-6f20-3fec-94cf-f7afa239f398"",""6fa8ad53-2f0d-3f44-8863-139092bfeda9""}]"
"""4c486414-d4f5-…","[{""016099ea-6f20-3fec-94cf-f7afa239f398"",""946e3ced-48a5-3de5-ad5a-1d20b1ab7eb5""}, {""a05a261f-128d-3cd8-a8e1-d6e52e161947"",""375c16ea-5f8d-32d5-8893-639d9b3a53d6""}, … {""68acdde8-bd53-39d1-9be0-fd67a281d7be"",""d7a53acc-eb6e-3f6c-b72e-9aefb54dd311""}]"
"""7e5838fc-bce1-…","[{""1f873432-6944-3df9-8300-8a3cf9f95b35"",""5862055b-35a6-316a-8e20-3ae20c1763c2""}, {""857c4b20-3057-30e0-9ca3-d6f5c3dbe4a6"",""857c4b20-3057-30e0-9ca3-d6f5c3dbe4a6""}, … {""016099ea-6f20-3fec-94cf-f7afa239f398"",""6fa8ad53-2f0d-3f44-8863-139092bfeda9""}]"


**Observation**
* Cleaned data already cotains engineered features

**Impact** 

* Do feature engineering on graph


In [45]:
callsProcessed = callGraph.with_columns(
    pl.col("call_graph").list.eval(
        pl.element().struct.rename_fields(["to", "from"])
    )
).explode("call_graph").unnest("call_graph")

In [46]:
callsProcessed

_id,to,from
str,str,str
"""1f2c32d8-2d6e-…","""1f873432-6944-…","""5862055b-35a6-…"
"""1f2c32d8-2d6e-…","""8955faa9-0e33-…","""a4fd6415-1fd4-…"
"""1f2c32d8-2d6e-…","""85754db8-6a55-…","""85754db8-6a55-…"
"""1f2c32d8-2d6e-…","""9f08fee1-953c-…","""876b4958-7df1-…"
"""1f2c32d8-2d6e-…","""857c4b20-3057-…","""857c4b20-3057-…"
…,…,…
"""468a84b3-2885-…","""d3fc2678-10cb-…","""d3fc2678-10cb-…"
"""468a84b3-2885-…","""7f68d54f-87ec-…","""7f68d54f-87ec-…"
"""2854b436-7d8b-…","""308d23af-c2ca-…","""2738e795-7c3e-…"
